In [ ]:
# %% [markdown]
# # 🎤 Simulation d'Entretien avec Groq (Llama 3.3 70B)
# 
# **Objectif** : Tester la génération de questions et l'évaluation des réponses
# 
# **Pipeline** :
# 1. Charger un CV et une offre
# 2. Générer des questions personnalisées
# 3. Simuler des réponses
# 4. Évaluer les réponses
# 5. Générer feedback global

In [4]:
# %%
import sys
sys.path.append('..')

import json
import os
from pathlib import Path
from src.interview_simulator import InterviewSimulator
from src.skills_extractor import SkillsExtractor
from src.cv_parser import CVParser

In [ ]:
# %%
# CONFIGURATION : Clé API Groq
# Remplacez par votre clé ou utilisez la variable d'environnement
os.environ["GROQ_API_KEY"] = "ta clé api ici"  # Remplacez par votre clé API Groq

# Ou si vous avez déjà défini la variable d'environnement :
# print(f"Clé API : {os.getenv('GROQ_API_KEY')[:20]}...")

# %%
# 1. INITIALISATION
print("="*60)
print("🚀 INITIALISATION")
print("="*60)

try:
    simulator = InterviewSimulator()
    skills_extractor = SkillsExtractor()
    print("✅ Tous les modules chargés")
except ValueError as e:
    print(f"❌ {e}")
    print("\n💡 Obtenez une clé gratuite sur : https://console.groq.com/")
    raise

🚀 INITIALISATION
✅ InterviewSimulator initialisé avec llama-3.3-70b-versatile
✅ Modèle spaCy chargé
✅ Base de compétences chargée
   • Techniques : 171
   • Soft skills : 39
✅ Tous les modules chargés


In [6]:
# %%
# 2. CHARGER UN CV ET UNE OFFRE
print("\n" + "="*60)
print("📂 CHARGEMENT DES DONNÉES")
print("="*60)

# Charger le CV
cv_path = Path("../data/RESUME Robert UNG.pdf")

if not cv_path.exists():
    print(f"⚠️  CV introuvable : {cv_path}")
    print("Utilisez votre propre CV pour tester")
    # Simuler des compétences
    cv_skills = ["Python", "Machine Learning", "TensorFlow", "Docker", "FastAPI", "Git"]
else:
    parser = CVParser()
    cv_text = parser.parse(str(cv_path))
    skills_result = skills_extractor.extract_from_cv(cv_text)
    cv_skills = skills_result['technical_skills']

print(f"\n✅ Compétences CV : {', '.join(cv_skills[:10])}")

# Charger une offre
with open('../data/jobs/jobs_dataset.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)
    job = dataset['jobs'][0]  # ML Engineer Junior

print(f"\n✅ Offre sélectionnée")
print(f"   Titre : {job['title']}")
print(f"   Entreprise : {job['company']}")
print(f"   Requirements : {', '.join(job['requirements'][:5])}...")


📂 CHARGEMENT DES DONNÉES

✅ Compétences CV : artificial intelligence, big data, c, data analysis, data science, deep learning, docker, excel, git, java

✅ Offre sélectionnée
   Titre : Junior ML Engineer
   Entreprise : AI Startup Paris
   Requirements : Python (numpy, pandas, scikit-learn), Machine Learning basics (supervised learning), Git et GitHub, Docker (notions de base), Anglais technique (lecture documentation)...


In [7]:
# %%
# 3. GÉNÉRATION DES QUESTIONS
print("\n" + "="*60)
print("❓ GÉNÉRATION DES QUESTIONS")
print("="*60)

questions = simulator.generate_questions(
    cv_skills=cv_skills,
    job_title=job['title'],
    job_description=job['description'],
    job_requirements=job['requirements'],
    num_questions=8
)

print(f"\n📋 QUESTIONS RH ({len(questions['rh_questions'])}):")
for q in questions['rh_questions']:
    print(f"\n{q['id']}. [{q['type']}]")
    print(f"   {q['question']}")

print(f"\n🔧 QUESTIONS TECHNIQUES ({len(questions['technical_questions'])}):")
for q in questions['technical_questions']:
    skill = q.get('skill', 'N/A')
    print(f"\n{q['id']}. [{q['type']} - {skill}]")
    print(f"   {q['question']}")


❓ GÉNÉRATION DES QUESTIONS
✅ 4 questions RH générées
✅ 4 questions techniques générées

📋 QUESTIONS RH (4):

1. [présentation]
   Parlez-moi de vos études et de votre parcours professionnel jusqu'à présent.

2. [motivation]
   Qu'est-ce qui vous motive à postuler pour ce poste de Junior ML Engineer ?

3. [soft_skills]
   Décrivez une situation où vous avez dû travailler en équipe pour résoudre un problème.

4. [projet]
   Pouvez-vous me parler d'un projet personnel ou scolaire qui vous a particulièrement intéressé ?

🔧 QUESTIONS TECHNIQUES (4):

5. [compétence_technique - Pandas]
   Comment utilisez-vous la bibliothèque Pandas pour l'analyse de données ?

6. [concept - Machine Learning]
   Expliquez les principes de base de l'apprentissage supervisé en Machine Learning.

7. [outil - Docker]
   Comment déployez-vous un modèle de Machine Learning en utilisant Docker ?

8. [compétence_technique - Scikit-learn]
   Quelle est votre expérience avec la bibliothèque Scikit-learn pour la class

In [8]:
# %%
# 4. SIMULATION DE RÉPONSES
print("\n" + "="*60)
print("💬 SIMULATION DE RÉPONSES")
print("="*60)

# Exemples de réponses (bonnes, moyennes, mauvaises)
simulated_answers = {
    1: """Je suis diplômé d'un Master en IA et j'ai développé plusieurs projets en Machine Learning pendant mes stages. 
    J'ai notamment travaillé sur un système de recommandation avec TensorFlow qui a amélioré l'engagement de 15%. 
    Je suis passionné par l'apprentissage automatique et cherche à approfondir mes compétences en production de modèles ML.""",
    
    2: """Je souhaite rejoindre votre entreprise car vous êtes leaders dans l'IA. J'ai suivi vos publications techniques 
    et je suis impressionné par vos innovations. Cette position me permettrait d'apprendre auprès d'experts tout en 
    contribuant à des projets impactants.""",
    
    5: """En Python, j'utilise pandas pour la manipulation de données. Pour nettoyer un dataset, je commence par 
    df.dropna() pour les valeurs manquantes, puis df.drop_duplicates(). J'utilise aussi df.describe() pour les statistiques 
    et df.groupby() pour les agrégations. Pour l'optimisation, je préfère les opérations vectorisées aux boucles.""",
    
    6: """Pour une API REST, je structurerais avec FastAPI : des endpoints clairs (/users, /posts), 
    validation Pydantic, gestion d'erreurs avec HTTPException, et documentation auto Swagger. 
    J'ajouterais aussi du rate limiting et de l'authentification JWT."""
}

print("✅ 4 réponses simulées prêtes pour évaluation")


💬 SIMULATION DE RÉPONSES
✅ 4 réponses simulées prêtes pour évaluation


In [9]:
# %%
# 5. ÉVALUATION DES RÉPONSES
print("\n" + "="*60)
print("📊 ÉVALUATION DES RÉPONSES")
print("="*60)

evaluations = []

# Évaluer les 4 premières questions
test_questions = questions['rh_questions'][:2] + questions['technical_questions'][:2]

for question in test_questions:
    q_id = question['id']
    
    if q_id not in simulated_answers:
        continue
    
    print(f"\n{'='*60}")
    print(f"❓ Question {q_id}: {question['question'][:80]}...")
    print(f"\n💬 Réponse candidat:")
    print(f"{simulated_answers[q_id][:150]}...")
    
    # Évaluer avec Groq
    evaluation = simulator.evaluate_answer(
        question=question['question'],
        answer=simulated_answers[q_id],
        question_type=question['type'],
        target_skill=question.get('skill')
    )
    
    evaluations.append(evaluation)
    
    print(f"\n📈 RÉSULTAT:")
    print(f"   Score: {evaluation['score']:.0f}/100")
    print(f"   Évaluation: {evaluation['evaluation'][:200]}...")
    
    print(f"\n✅ Points forts:")
    for point in evaluation['points_forts'][:3]:
        print(f"   • {point}")
    
    print(f"\n⚠️  À améliorer:")
    for point in evaluation['points_amelioration'][:3]:
        print(f"   • {point}")


📊 ÉVALUATION DES RÉPONSES

❓ Question 1: Parlez-moi de vos études et de votre parcours professionnel jusqu'à présent....

💬 Réponse candidat:
Je suis diplômé d'un Master en IA et j'ai développé plusieurs projets en Machine Learning pendant mes stages. 
    J'ai notamment travaillé sur un sys...
✅ Réponse évaluée : 80/100

📈 RÉSULTAT:
   Score: 80/100
   Évaluation: Réponse claire et structurée, démontrant une bonne compréhension de son parcours et de ses compétences en IA et Machine Learning....

✅ Points forts:
   • Exemples concrets mentionnés, tels que le système de recommandation avec TensorFlow
   • Bonne structure de réponse, facile à suivre
   • Démontre une passion pour l'apprentissage automatique et une volonté d'approfondir ses compétences

⚠️  À améliorer:
   • Pourrait ajouter plus de détails sur son rôle dans les projets et les défis rencontrés
   • Manque de diversité dans les exemples présentés, presque exclusivement axés sur le Machine Learning

❓ Question 2: Qu'est-ce

In [10]:
# %%
# 6. FEEDBACK GLOBAL
print("\n" + "="*60)
print("🎯 FEEDBACK GLOBAL")
print("="*60)

global_feedback = simulator.generate_final_feedback(
    evaluations=evaluations,
    job_title=job['title']
)

print(f"\n📊 Score global: {global_feedback['global_score']:.1f}/100")
print(f"🎯 Décision: {global_feedback['decision']}")

print(f"\n📝 Synthèse:")
print(f"{global_feedback['synthese']}")

print(f"\n✅ Compétences validées:")
for comp in global_feedback.get('competences_validees', [])[:5]:
    print(f"   • {comp}")

print(f"\n📈 Axes de progression:")
for axe in global_feedback.get('axes_progression', [])[:5]:
    print(f"   • {axe}")

print(f"\n🚀 Prochaines étapes:")
for etape in global_feedback.get('prochaines_etapes', [])[:5]:
    print(f"   • {etape}")


🎯 FEEDBACK GLOBAL
✅ Feedback global généré

📊 Score global: 60.0/100
🎯 Décision: Prometteur

📝 Synthèse:
Le candidat a démontré une motivation pour apprendre et contribuer, ainsi qu'une bonne compréhension des principes de base de Pandas et de l'apprentissage automatique. Cependant, il doit approfondir ses connaissances et fournir des exemples concrets avec des résultats mesurables. Avec un peu de travail, le candidat a le potentiel pour devenir un excellent Junior ML Engineer.

✅ Compétences validées:
   • Motivation pour apprendre et contribuer
   • Compréhension des principes de base de Pandas
   • Connaissances en apprentissage automatique
   • Structure de réponse claire et facile à suivre
   • Passion pour l'apprentissage automatique

📈 Axes de progression:
   • Approfondir les connaissances en Machine Learning
   • Fournir des exemples concrets avec des résultats mesurables
   • Développer des compétences en gestion de données manquantes
   • Élargir la diversité des exemples p

In [ ]:
# %% [markdown]
# ## ✅ Résumé
# 
# - ✅ **Groq (Llama 3.3 70B)** : Génération de questions ultra-rapide
# - ✅ **Questions personnalisées** : Basées sur CV + offre
# - ✅ **Évaluation automatique** : Scoring 0-100 avec feedback
# - ✅ **Feedback global** : Synthèse + recommandations
# 
# **Prochaine étape** : Intégrer dans l'API et Streamlit